# Compile Script

We compile the shell script for experiments

In [1]:
exprs = [
    # basic numeric
    "df.sum()",
    "df.sum(axis=1)",
    "df.median()",
    "df.median(axis=1)",
    "df.std()",
    "df.std(axis=1)",
    # transpose
    "wait(df.T)",
    # apply 
    "wait(df.apply(lambda x:x))",
    "df.apply('sum')",
    "df.apply('median')",
    "df.apply('std')",
    # sort
    "wait(df.sort_values(df.columns[0]))",
    "wait(df.sort_values(df.columns))",
    "df.idxmin()",
    # nunique
    "df.nunique()",
    "df.nunique(axis=1)"
]

In [2]:
runners = ['ray', 'pandas']

In [3]:
data = [
    'data/coarse_tall.pq',
    'data/coarse_wide.pq',
    'data/fine_tall.pq'
]

In [4]:
num_cpus = [1,4,8,16,32,64]

In [5]:
import pandas as pd

In [40]:
from itertools import product, chain

In [9]:
all_combination = list(product(exprs, runners, data, num_cpus))

In [16]:
df = pd.DataFrame.from_records(all_combination, columns=['--expr', '--runner', '--data', '--num-cpus'])

In [17]:
df.head()

,--expr,--runner,--data,--num-cpus
0,df.sum(),ray,data/coarse_tall.pq,1
1,df.sum(),ray,data/coarse_tall.pq,4
2,df.sum(),ray,data/coarse_tall.pq,8
3,df.sum(),ray,data/coarse_tall.pq,16
4,df.sum(),ray,data/coarse_tall.pq,32


In [36]:
df['--expr'] = df['--expr'].apply(lambda x:f'"{x}"')

In [37]:
final = df[~((df['--runner'] == 'pandas') & (df['--num-cpus'] != 1))]

In [38]:
f = next(df.iterrows())[1]

In [47]:
with open('run.sh','w') as f:
    for idx, series in df.iterrows():
        cmd = ' '.join(map(str, chain.from_iterable(series.iteritems())))
        f.write(f"python runner.py {cmd} --log-dir logs\n")